In [3]:
import yaml

with open("few_shot_prompts.yaml", "r") as yaml_file:
    loaded_yaml = yaml.safe_load(yaml_file)

In [4]:
for key, value in loaded_yaml.items():
    print("Question: ", key)
    print("Code: ", value)
    break

Question:  List all "My Brands" or what brands I own
Code:   # Filter for rows where MyBrand is 1
df_my_brands = data[data['MyBrand'] == 1]['Brand'].unique()
df_my_brands 


In [5]:
import pandas as pd 

data = pd.read_csv("summary.csv")

In [8]:
count = 7
ind = 0
for key, value in loaded_yaml.items():
    ind+=1
    if ind==count:
        print("Question: ", key)
        print("Code: ", value)
        exec(value)
        display(eval(value.split('\n')[-1]))
    else: 
        pass

Question:  How can Neutrogena improve its equity score? (show worst performing dimensions and competitors)
Code:  
# Find worst performing dimensions and compare equity score
df_filtered = data[data['Detailed Metric'] == 'Beat Score']
df_worst = df_filtered[df_filtered['Brand'] == 'Neutrogena'].sort_values('Score').head()
df_worst



SyntaxError: invalid syntax (<string>, line 0)

In [9]:
df_worst

,Category,MyBrand,Brand,Prestige,Brand Pulse Component,Detailed Metric,Month_Start_Date,Year_Month,Year,Score,Score_Type,Channel,TotalConversations
2020,Skin Care US,1,Neutrogena,NaN,Buzz,Beat Score,2022-09-01,2022_09 - September,2022,33.851,Beat Score,Mixed,15147
5796,Skin Care US,1,Neutrogena,NaN,Buzz,Beat Score,2022-11-01,2022_11 - November,2022,34.078,Beat Score,Mixed,16313
3913,Skin Care US,1,Neutrogena,NaN,Buzz,Beat Score,2022-10-01,2022_10 - October,2022,34.103,Beat Score,Mixed,15967
139,Skin Care US,1,Neutrogena,NaN,Buzz,Beat Score,2022-08-01,2022_08 - August,2022,34.306,Beat Score,Mixed,17222
7677,Skin Care US,1,Neutrogena,NaN,Buzz,Beat Score,2022-12-01,2022_12 - December,2022,35.976,Beat Score,Mixed,42841


In [7]:
data

,Category,MyBrand,Brand,Prestige,Brand Pulse Component,Detailed Metric,Month_Start_Date,Year_Month,Year,Score,Score_Type,Channel,TotalConversations
0,Skin Care US,0,Shea Moisture,NaN,Action,Beat Score,2022-08-01,2022_08 - August,2022,64.162,Beat Score,Mixed,243
1,Skin Care US,0,Hawaiian Tropic,NaN,Action,Beat Score,2022-08-01,2022_08 - August,2022,35.558,Beat Score,Mixed,20
2,Skin Care US,0,La Roche-Posay,NaN,Action,Beat Score,2022-08-01,2022_08 - August,2022,64.105,Beat Score,Mixed,507
3,Skin Care US,0,Garnier,NaN,Action,Beat Score,2022-08-01,2022_08 - August,2022,70.506,Beat Score,Mixed,233
4,Skin Care US,0,Gold Bond,NaN,Action,Beat Score,2022-08-01,2022_08 - August,2022,57.731,Beat Score,Mixed,104
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43235,Skin Care US,0,StriVectin,NaN,Trust,Social_NPS,2024-06-01,2024_06 - June,2024,89.189,Sub Score,Social,49
43236,Skin Care US,0,Vichy,NaN,Trust,Social_NPS,2024-06-01,2024_06 - June,2024,77.344,Sub Score,Social,55
43237,Skin Care US,0,Murad,NaN,Trust,Social_NPS,2024-06-01,2024_06 - June,2024,79.048,Sub Score,Social,27
43238,Skin Care US,1,Neostrata,NaN,Trust,Social_NPS,2024-06-01,2024_06 - June,2024,72.727,Sub Score,Social,8


In [5]:
from haystack import Document, Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.rankers import SentenceTransformersDiversityRanker

docs = [Document(content="The iconic Eiffel Tower is a symbol of Paris"),
        Document(content="Visit Luxembourg Gardens for a haven of tranquility in Paris"),
        Document(content="The Pont Alexandre III bridge in Paris is famous for its Beaux-Arts style")]
document_store = InMemoryDocumentStore()
document_store.write_documents(docs)

retriever = InMemoryBM25Retriever(document_store = document_store)
ranker = SentenceTransformersDiversityRanker()

document_ranker_pipeline = Pipeline()
document_ranker_pipeline.add_component(instance=retriever, name="retriever")
document_ranker_pipeline.add_component(instance=ranker, name="ranker")

document_ranker_pipeline.connect("retriever.documents", "ranker.documents")

query = "Most famous iconic sight in Paris"
document_ranker_pipeline.run(data={"retriever": {"query": query, "top_k": 3}, 
                                   "ranker": {"query": query, "top_k": 2}})

{'ranker': {'documents': [Document(id=814313dd9eed5ed295679a2cfeb58ebd2fba509befc417726e5e289226e970df, content: 'The iconic Eiffel Tower is a symbol of Paris', score: 2.2886661933209442),
   Document(id=5baeb730365df17da3c151fdc3c3b445c8c1f2d16cc2b1ba6e754f0b84150f62, content: 'Visit Luxembourg Gardens for a haven of tranquility in Paris', score: 1.9523053892242035)]}}